##Imports, Downloading the data and cleaning
We use the dataset from huggingface


In [1]:
#Requirements for colab
#!pip install transformers hackernews-client datasets -q

     |████████████████████████████████| 3.5 MB 5.2 MB/s 
     |████████████████████████████████| 311 kB 35.2 MB/s 
     |████████████████████████████████| 596 kB 16.5 MB/s 
     |████████████████████████████████| 6.8 MB 27.9 MB/s 
     |████████████████████████████████| 895 kB 17.5 MB/s 
     |████████████████████████████████| 67 kB 4.0 MB/s 
     |████████████████████████████████| 1.1 MB 30.5 MB/s 
     |████████████████████████████████| 243 kB 34.2 MB/s 
     |████████████████████████████████| 133 kB 38.7 MB/s 
     |████████████████████████████████| 271 kB 46.7 MB/s 
     |████████████████████████████████| 94 kB 1.8 MB/s 
     |████████████████████████████████| 144 kB 40.6 MB/s 


# Topic classification
## Downloading and cleaning data

In [2]:
import pandas as pd
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import StratifiedShuffleSplit
SEED = 42

In [3]:
dataset = load_dataset("Fraser/news-category-dataset")

Downloading:   0%|          | 0.00/5.53k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/23.4M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset news_category downloaded and prepared to /root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb9c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
reduced_categories = {
  "CULTURE & ARTS":["ARTS","ARTS & CULTURE","CULTURE & ARTS",
                    "COMEDY","ENTERTAINMENT","MEDIA"],
 "EDUCATION": ["EDUCATION","COLLEGE"],
 "BUSINESS": ["BUSINESS","MONEY"],
 "HEALTH & LIVING": ["WELLNESS","HEALTHY LIVING","TRAVEL","IMPACT","FIFTY",
                     "STYLE & BEAUTY","HOME & LIVING","GREEN",
                     "PARENTS","STYLE","FOOD & DRINK","TASTE","PARENTING","DIVORCE","WEDDINGS"],
 "SPORTS" : ["SPORTS"],
 "NEWS & POLITICS" :["POLITICS","BLACK VOICES","LATINO VOICES","QUEER VOICES","WOMEN","RELIGION"]+["GOOD NEWS","THE WORLDPOST","WORLDPOST","WORLD NEWS","WEIRD NEWS","CRIME"],
 "TECH & SCIENCE": ["SCIENCE","ENVIRONMENT","TECH"]
 }
 
label_map = {}
for i,cats in enumerate(reduced_categories.values()):
  for cat in cats:
    label_map[cat] = i

In [5]:
num_labels = len(reduced_categories.values())

In [6]:
#combinne train and test for
#headline + short_description
#headline
#short_description
combined_dataset = pd.DataFrame(
    {
        "text" : 
     ((pd.Series(dataset["train"]["headline"]) +\
                   "\n" + pd.Series(dataset["train"]["short_description"]))
        .append((pd.Series(dataset["test"]["headline"]) +\
                 "\n" + pd.Series(dataset["test"]["short_description"]))))
        .append((pd.Series(dataset["train"]["short_description"]))
        .append( pd.Series(dataset["test"]["short_description"])))
        .append(
            pd.Series(dataset["train"]["headline"])
            .append(pd.Series(dataset["test"]["headline"]))
            )
        ,
        "label" :
     pd.Series(dataset["train"]["category"]).replace(label_map)
        .append(pd.Series(dataset["test"]["category"]).replace(label_map))
        .append(pd.Series(dataset["train"]["category"]).replace(label_map))
        .append(pd.Series(dataset["test"]["category"]).replace(label_map))
        .append(pd.Series(dataset["train"]["category"]).replace(label_map)
        .append(pd.Series(dataset["test"]["category"]).replace(label_map)))
    }

)

In [9]:
combined_dataset = combined_dataset.drop_duplicates(subset=['text'])
# We want to balance the dataset
combined_dataset = combined_dataset.groupby("label").sample(
    combined_dataset.label.value_counts().min())

In [11]:
ssplit=StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_index, test_index = next(ssplit.split(combined_dataset.text,
                                           combined_dataset.label))

In [12]:

train_dataset = Dataset.from_dict({
    "text" : combined_dataset.iloc[train_index].text,
    "label": combined_dataset.iloc[train_index].label
})


test_dataset = Dataset.from_dict({
    "text" : combined_dataset.iloc[test_index].text,
    "label": combined_dataset.iloc[test_index].label
})

In [13]:
train_dataset.save_to_disk("./tmp/train_dataset")
test_dataset.save_to_disk("./tmp/test_dataset")

## Training the model

In [14]:
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
from datasets import load_metric
import numpy as np
metric = load_metric("accuracy")

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

In [15]:
train_dataset = Dataset.from_file("./tmp/train_dataset/dataset.arrow")
test_dataset  = Dataset.from_file("./tmp/test_dataset/dataset.arrow")


In [16]:
model_checkpoint = "microsoft/xtremedistil-l6-h256-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/525 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/xtremedistil-l6-h256-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
def preprocess_function(examples, tokenizer):
    return tokenizer(examples["text"],
                   padding="max_length",max_length=201 ,
                   truncation=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
encoded_train_dataset = train_dataset.map(lambda x: preprocess_function(x,tokenizer),batched=True)
encoded_test_dataset =  test_dataset.map(lambda x: preprocess_function(x,tokenizer),batched=True)

  0%|          | 0/33 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [25]:
print("Cat:",encoded_test_dataset["label"][0],
      "\n",encoded_test_dataset["text"][0],"\n",
      list(reduced_categories.values())[encoded_test_dataset["label"][0]])

Cat: 3 
 Dwyane Wade Divorce Fees Could Reach $10 Million: Report
Wade and Funches-Wade were married in 2002 and have two sons together, Zion and Zaire. Wade filed for divorce in 2007, and 
 ['WELLNESS', 'HEALTHY LIVING', 'TRAVEL', 'IMPACT', 'FIFTY', 'STYLE & BEAUTY', 'HOME & LIVING', 'GREEN', 'PARENTS', 'STYLE', 'FOOD & DRINK', 'TASTE', 'PARENTING', 'DIVORCE', 'WEDDINGS']


In [21]:
metric_name = "accuracy"
batch_size= 16
args = TrainingArguments(
    f"finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
)

In [26]:

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("best_model")

Saving model checkpoint to best_model
Configuration saved in best_model/config.json
Model weights saved in best_model/pytorch_model.bin
tokenizer config file saved in best_model/tokenizer_config.json
Special tokens file saved in best_model/special_tokens_map.json


## Interference


In [ ]:
model_test = AutoModelForSequenceClassification.from_pretrained(
    f"./best_model/",
    num_labels=num_labels)


#source: https://github.com/huggingface/transformers/blob/master/src/transformers/pipelines/text_classification.py
def softmax(_outputs):
    maxes = np.max(_outputs, axis=-1, keepdims=True)
    shifted_exp = np.exp(_outputs - maxes)
    return shifted_exp / shifted_exp.sum(axis=-1, keepdims=True)

def test_interference(trained_model, text, tokenizer):
    text_pt = tokenizer([text],
                    padding="max_length",max_length=201,
                    truncation=True,return_tensors="pt")
    return softmax(trained_model(**text_pt)[0][0].detach().numpy())

loading configuration file ./best_model/config.json
Model config BertConfig {
  "_name_or_path": "./best_model/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  }

In [ ]:
list(reduced_categories.values())[np.argmax(test_interference(model_test, "A new beer was introduced today", tokenizer))]

['WELLNESS',
 'HEALTHY LIVING',
 'STYLE & BEAUTY',
 'HOME & LIVING',
 'PARENTS',
 'STYLE',
 'FOOD & DRINK',
 'TASTE',
 'PARENTING',
 'DIVORCE',
 'WEDDINGS']

## Fetch Hackernews
Fetch headline and first comment

In [ ]:
from hackernews import hn
news_client = hn.NewsClient()

def get_comm(x):
  try:
    c_id = x.kids[0]
    return requests.get(f"https://hacker-news.firebaseio.com/v0/item/{c_id}.json?print=pretty").json()['text']
  except:
    return ""

h = news_client.get_best_story(fetchMax=200)

headline_and_comment  =  [x.title + " " + get_comm(x) for x in h]


## Test Hacker News Model



In [ ]:
headline_and_comment[26]

"The Banality of Genius: Notes on Peter Jackson's Get Back "

In [ ]:
lookup_label[np.argmax(test_interference(model_test,headline_and_comment[26]))]

['COMEDY', 'ENTERTAINMENT', 'MEDIA']